In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df_aep = pd.read_csv("data/AEP_hourly.csv")
df_aep.rename(columns={'Datetime': 'ds', 'AEP_MW': 'y'}, inplace=True)
df_aep['ds'] = pd.to_datetime(df_aep['ds'], format='%Y-%m-%d %H:%M:%S')
out_dir = 'uncertainty_results'

In [8]:
import RNN_fourier_RNN_uncertainty
import pickle

train_config = RNN_fourier_RNN_uncertainty.training_config(n_epochs=30, device=torch.device("mps"))
fourier_conf = RNN_fourier_RNN_uncertainty.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)

K_total = fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly
F_per_hour = 2 * K_total

if fourier_conf.mode == "vector":
    cont_dim = 1 + F_per_hour
    fourier_dim = F_per_hour
else:
    cont_dim = 1 + 24 * F_per_hour
    fourier_dim = F_per_hour

results_fourier = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
train_residuals_results = {}
epsilon_params_results = {}
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy()
    model = RNN_fourier_RNN_uncertainty.RNN_fourier(cont_dim=cont_dim, fourier_dim=fourier_dim, xf_mode="matrix",
                                                    d_model=128,latent_dim=32,nhead=4)
    trainer = RNN_fourier_RNN_uncertainty.RNN_train_fourier(model, train_config, fourier_conf)
    train_results = trainer(df_reduced)

    forcast = train_results['test_pred']
    true = train_results['test_true']
    train_residuals = train_results['train_residuals']
    epsilon_params = {'mean': train_results['epsilon_mean'], 'cov': train_results['epsilon_cov']}

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})
    results_fourier = pd.concat([results_fourier, result]).reset_index(drop=True)
    train_residuals_results[str(df_reduced['ds'].dt.date.max())] = train_residuals
    epsilon_params_results[str(df_reduced['ds'].dt.date.max())] = epsilon_params

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i % 5 == 0:
        results_fourier.to_csv('experiment_results/' + out_dir + '/results_fourier.csv', index=False)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_fourier.to_csv('experiment_results/' + out_dir + '/results_fourier.csv', index=False)

with open('experiment_results/' + out_dir + '/train_residuals_results.pkl', 'wb') as f:
    pickle.dump(train_residuals_results, f)

with open('experiment_results/' + out_dir + '/epsilon_params_results.pkl', 'wb') as f:
    pickle.dump(epsilon_params_results, f)


1
epoch 1 loss: 0.5938
epoch 2 loss: 0.1502
epoch 3 loss: 0.0950


/var/folders/3h/5yfj7yqx7lg98t310bfxxr3m0000gn/T/ipykernel_13591/1381903466.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_fourier = pd.concat([results_fourier, result]).reset_index(drop=True)


2
epoch 1 loss: 0.5081
epoch 2 loss: 0.2683
epoch 3 loss: 0.1803
3
epoch 1 loss: 0.4033
epoch 2 loss: 0.2543
epoch 3 loss: 0.1598
4
epoch 1 loss: 0.4337
epoch 2 loss: 0.1322
epoch 3 loss: 0.0844
5
epoch 1 loss: 0.3931
epoch 2 loss: 0.1913
epoch 3 loss: 0.0918
finish
